
# Keras

The most convenient way to use TensorFlow with neural networks is through [Keras](http://keras.io). It provides a high-level interface that is somewhat a compromise between very high-level abstractions like scikit-learn and the complete control of every detail you get when directly using the low-level APIs of libraries like TensorFlow.

Besides TensorFlow, Keras also supports [Theano](http://www.deeplearning.net/software/theano/) and [CNTK](https://docs.microsoft.com/en-us/cognitive-toolkit/). Code that only uses the high-level Keras API should be able to run in Theano and CNTK as well by just switching the backend. If you only want to use TensorFlow, you can use the version of keras that is directly included in TensorFlow (`tensorflow.keras`)

## Build a model in Keras

In [ ]:
import keras

As a quick example, let's again build a model to classify our "Moons" dataset from [NNFromScratch.ipynb](NNFromScratch.ipynb).

In [ ]:
from sklearn.datasets import make_moons

In [ ]:
x, y = make_moons(n_samples=10000, noise=0.2)

There are 2 main ways to use Keras - via the Sequential or the Functional API. Lets start with `Sequential`. This is convenient for all models where we just have one input and one output Tensor with stacked Layers in between. Here we use the `Dense` layer - which is precisely the fully connected NN layer that applies the $\sigma(W\mathbf{x} + \mathbf{b})$ operation.

In [ ]:
from keras.layers import Dense

model = keras.models.Sequential([
    # Hidden layer with 2 inputs, 16 outputs
    Dense(16, activation="relu", input_shape=(2,)),
    # Output layer with 16 inputs (determined automatically) and 1 output
    Dense(1, activation="sigmoid")
])

How much parameters will our model have? The answer:

In [ ]:
model.summary()

We can also access the underlying Tensors if needed:

In [ ]:
model.inputs

In [ ]:
model.outputs

In [ ]:
model.weights

In [ ]:
model.layers

In [ ]:
model.layers[0].input

In [ ]:
model.layers[0].output

In [ ]:
%load_ext tensorboard
import tensorflow as tf

In [ ]:
from tensorflow.python.summary.writer.writer import FileWriter
FileWriter('logs/train', graph=tf.get_default_graph()).close()
%tensorboard --logdir logs/train

## Train the model

Before we can run the training, we have to "compile" the model. This will configure the loss function and optimization Algorithm. You cat pass each loss from [`keras.losses`](https://keras.io/losses) and each optimizer from [`keras.optimizers`](https://keras.io/optimizers) also as a string with the name if you want to use it with default parameters. Here we want to use the "Adam" optimizer with an adjusted initial learning rate, so we pass it directly.

We could also pass some metrics that we want to monitor during training (in addition to the Loss value).

In [ ]:
model.compile(optimizer=keras.optimizers.Adam(lr=0.1), loss="binary_crossentropy")

The API for fitting looks similar to scikit-learn, but has additional options. In fact there also is a [scikit-learn API  wrapper](https://keras.io/scikit-learn-api/) for Keras if you need that in some context.

In [ ]:
history = model.fit(x, y, epochs=3, batch_size=128)

The returned `History` object contains the monitored metrics (by default the loss).

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(history.epoch, history.history['loss'])

## Run the model

The model can be run using `model.predict`.

In [ ]:
import numpy as np

In [ ]:
grid = np.meshgrid(
    np.arange(x[:,0].min(), x[:,0].max(), 0.1),
    np.arange(x[:,1].min(), x[:,1].max(), 0.1),
)

In [ ]:
xy = np.stack([grid[0].ravel(), grid[1].ravel()], axis=1)
xy

In [ ]:
scores = model.predict(xy)

In [ ]:
plt.contourf(grid[0], grid[1], scores.reshape(grid[0].shape), cmap="Spectral_r")
plt.colorbar(label="NN output")
opts = dict(alpha=0.1, marker=".", edgecolors="black")
plt.scatter(x[y==0][:,0], x[y==0][:,1], color="blue", **opts)
plt.scatter(x[y==1][:,0], x[y==1][:,1], color="red", **opts)
plt.xlim(grid[0].min(), grid[0].max())
plt.ylim(grid[1].min(), grid[1].max())

## Functional API

The functional API is very similar to the low-level TensorFlow API. Each layer can be called as a function on an input Tensor and return an output Tensor. One can then build arbitrary computation graphs and finally build a model by passing the input and output Tensors. This is especially useful when we want to organize the processing into different inputs and different outputs or if you want to build computation graphs that have branches.

Suppose we want to do some strangely complicated processing of the "California housing dataset":

In [ ]:
from sklearn.datasets import california_housing

In [ ]:
data = california_housing.fetch_california_housing()

In [ ]:
print(data.DESCR)

For convenience, let's put it into a DataFrame

In [ ]:
import pandas as pd
df_housing = pd.DataFrame(data.data)
df_housing.columns = data.feature_names
df_housing['MedHouseVal'] = data.target

In [ ]:
df_housing.head()

In [ ]:
df_housing.describe()

Lets do the following funny exercise:
* Feed the Latitude and Longitude through a separate NN layer
* Combine the output of this layer with the other inputs (except for the median income)
* Add another hidden layer
* Add a target where we first try to predict the median income
* Feed back this predicted median income  together with the outputs of the NN into another hidden layer
* Finally predict the median house value

In [ ]:
from keras.layers import Input, Dense, concatenate

# For such more complicated structures it is often useful to give the layers names

inp_feat = Input((5,), name="Features") # ['HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup']
inp_coord = Input((2,), name="Coordinates") # ['Latitude', 'Longitude']
hl_coord = Dense(64, activation="relu")(inp_coord)
joined_inp = concatenate([inp_feat, hl_coord])
hl = Dense(64, activation="relu")(joined_inp)
# no activation function here, this will be a regression target
out_MedInc = Dense(1, name="MedIncOutput")(hl)
joined_inp2 = concatenate([hl, out_MedInc])
hl2 = Dense(64, activation="relu")(joined_inp2)
out_HouseValue = Dense(1, name="HouseValueOutput")(hl2)

So we have a model now with 2 inputs and 2 outputs. We can use `keras.models.Model` to create models with arbitrary many inputs and outputs.

In [ ]:
housing_model = keras.models.Model(inputs=[inp_feat, inp_coord], outputs=[out_MedInc, out_HouseValue])
housing_model.summary()

In [ ]:
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

SVG(model_to_dot(housing_model).create(prog='dot', format='svg'))

Since we named the inputs and outputs, we can give input and target data as dictionaries, but before that we want to standardize both the inputs and the targets!

In [ ]:
features = ['HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup']

In [ ]:
coordinates = ['Latitude', 'Longitude']

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
scaler = StandardScaler().fit(df_housing.values)

In [ ]:
df_trf = df_housing.copy()
df_trf[:] = scaler.transform(df_housing.values)
df_trf.describe()

In [ ]:
x_housing = {
    "Features" : df_trf[features].values,
    "Coordinates" : df_trf[coordinates].values,
}
y_housing = {
    "MedIncOutput" : df_trf["MedInc"].values.reshape(-1, 1),
    "HouseValueOutput" : df_trf["MedHouseVal"].values.reshape(-1, 1),
}

We need to specify loss functions for all outputs. If different outputs should be trained with different loss functions, you need to pass a list. The total loss will be the sum of the individual losses. One could also pass `loss_weights` to weight them relative to each other, but we don't do that here.

In [ ]:
housing_model.compile(loss='mean_squared_error', optimizer='Adam')

In [ ]:
housing_model.fit(x_housing, y_housing, epochs=10, shuffle=True, batch_size=128)

Did we predict the median income and finally the house price correctly? Let's have a look at the distributions for true and predicted values.

In [ ]:
predictions = housing_model.predict(x_housing)

In [ ]:
opt = dict(alpha=0.5, bins=100, range=(-3, 5))
plt.hist(df_trf["MedInc"], label="True", **opt)
plt.hist(predictions[0], label="Predicted", **opt)
plt.xlabel("Median income (rescaled)")
plt.legend()

In [ ]:
opt = dict(alpha=0.5, bins=100, range=(-3, 5))
plt.hist(df_trf["MedHouseVal"], label="True", **opt)
plt.hist(predictions[1], label="Predicted", **opt)
plt.xlabel("Median House value (rescaled)")
plt.legend()

## Generators

Sometimes the whole training data might not fit into memory or you might want to do some live pre-processing. The simplest way to do this is via [python generators](https://wiki.python.org/moin/Generators)

Let's write a generator that yields an infinite amount of mini batches for our "moon" dataset. The generator should yield batches of (x, y).

In [ ]:
def moon_generator(batch_size=128, buffer_size=10000):
    # let's make an infinite generator
    # - in each pass of the loop we will generate `buffer_size` training examples
    while True:
        x, y = make_moons(n_samples=buffer_size, noise=0.4)
        # this is the loop over mini-batches
        for start in range(0, buffer_size, batch_size):
            yield x[start : start + batch_size], y[start : start + batch_size]

Let's make an overly complicated model and train it with "infinite data"

In [ ]:
stupid_model = keras.models.Sequential([
    Dense(1024, activation="relu", input_shape=(2,)),
    Dense(1024, activation="relu"),
    Dense(1024, activation="relu"),
    Dense(1, activation="sigmoid")
])

In [ ]:
stupid_model.summary()

In [ ]:
stupid_model.compile(optimizer="Adam", loss="binary_crossentropy")

We can use the [`fit_generator`](https://keras.io/models/model/#fit_generator) method to train the model. Since our generator is infinite we have to pass the `steps_per_epoch` Argument that defines how many batches should be used until one epoch is declared finished.

In [ ]:
stupid_model.fit_generator(moon_generator(), steps_per_epoch=200, epochs=1)

In [ ]:
def validate_with_generator(model, generator, steps=5):

    # for plotting, just draw a few examples from the generator
    x = []
    y = []
    for i in range(steps):
        data = next(generator)
        x.append(data[0])
        y.append(data[1])
    x = np.concatenate(x)
    y = np.concatenate(y)
    
    grid = np.meshgrid(
        np.arange(x[:,0].min(), x[:,0].max(), 0.1),
        np.arange(x[:,1].min(), x[:,1].max(), 0.1),
    )
    
    xy = np.stack([grid[0].ravel(), grid[1].ravel()], axis=1)    
    scores = model.predict(xy)

    plt.contourf(grid[0], grid[1], scores.reshape(grid[0].shape), cmap="Spectral_r")
    plt.colorbar(label="NN output")
    opts = dict(alpha=0.2, marker=".", edgecolors="black")
    plt.scatter(x[y==0][:,0], x[y==0][:,1], color="blue", **opts)
    plt.scatter(x[y==1][:,0], x[y==1][:,1], color="red", **opts)
    plt.xlim(grid[0].min(), grid[0].max())
    plt.ylim(grid[1].min(), grid[1].max())

In [ ]:
validate_with_generator(stupid_model, moon_generator(), steps=20)

## Backend API

While it is very well possible to operate on the underlying Tensors with TensorFlow you can also do this via the `keras.backend` API. This module wraps the around all operations to provide the mapping to the different backends (Thensorflow, Theano, CNTK). In most examples you find people will call this module `K`.

In [ ]:
import keras.backend as K

Let's try to visualize the hidden layers of our first neural network in this notebook

In [ ]:
model.summary()

In [ ]:
model.get_layer('dense_1').output

In [ ]:
model.input

`K.function` creates a function for us that we can feed a numpy array later. The following function will give us the output vector of the hidden layer.

In [ ]:
f = K.function([model.input], [model.get_layer('dense_1').output])

Let's feed it with a regular grid again for visualization.

In [ ]:
step = 0.1
grid = np.meshgrid(
    np.arange(x[:,0].min(), x[:,0].max()+step, step),
    np.arange(x[:,1].min(), x[:,1].max()+step, step)
)

In [ ]:
xp = np.c_[grid[0].ravel(), grid[1].ravel()]

In [ ]:
hl_out = f([xp])[0]

In [ ]:
fig, axs = plt.subplots(nrows=4, ncols=4, figsize=(10, 10))
for i in range(16):
    axs.ravel()[i].contourf(grid[0], grid[1], hl_out[:,i].reshape(grid[0].shape))

This gives a nice idea about how a NN composes it's output by combining the outputs of the previous layer. A nice visualization of this can be seen at https://playground.tensorflow.org/